# [NML-24] Assignment 2: Graph Neural Networks

TAs: [William Cappelletti](https://people.epfl.ch/william.cappelletti) and [Abdellah Rahmani](https://people.epfl.ch/abdellah.rahmani)

## Students

* Team: `<your team number>`
* Students: `<student1>`, `<student2>`

## Instructions

**!! Read carefully before starting !!**

**Deadline:** April 30

**Grading:**
* The integrality of Assignment 2 will be scaled to 100% and will amount to 1/3 of the overall assignments score.
* The total number of points is **100**, the points for each exercise are stated in the instructions.
* All team members will receive the same grade based on the team solution.
* Collaboration between team members is encouraged. No collaboration between teams is allowed.

**Expected output:**

You will have coding and theoretical questions. Coding exercises shall be solved within the specified space:
```python
# Your solution here ###########################################################
...
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
```
Sometimes we provide variable names, such as `x = ...`; do not change names and stick to hinted typing, as they will be reused later.
Within the solution space, you can declare any other variable or function that you might need, but anything outside these lines shall not be changed, or it will invalidate your answers.

Theoretical questions shall be answered in the following markdown cell. The first line will be
```markdown
**Your answer here:**
...
```

**Solutions:**
* Your submission is self-contained in the `.ipynb` file.

* Code has to be clean and readable. Provide meaningful variable names and comment where needed.

* Textual answers in [markdown cells][md_cells] shall be short: one to two
  sentences. Math shall be written in [LaTeX][md_latex].
    **NOTE**: handwritten notes pasted in the notebook are ignored.

* You cannot import any other library than we imported, unless explicitly stated.

* Make sure all cells are executed before submitting. I.e., if you open the notebook again it should show numerical results and plots. Cells not run are ignored.

* Execute your notebook from a blank state before submission, to make sure it is reproducible. You can click "Kernel" then "Restart Kernel and Run All Cells" in Jupyter. We might re-run cells to ensure that the code is working and corresponds to the results.

[md_cells]: https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html
[md_latex]: https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html#LaTex-equations

## Objective

This assignment focuses on Graph Neural Networks. In the first part, you will load and prepare data using the PyTorch Geometric library. Next, you will define a GNN used to solve a task. You will train and test the Neural Network and comment on the results.
In the second part, you will define a new GNN block, in order to include it in the previous architecture.
The third part is theoretical and study a way to include structural properties in learned networks.

## Prerequisites

The additional [tutorial notebook](nml24_gnn_tutorial.ipynb) provides a broad overview of PyTorch and PyTorch Geometric, showing how to manipulate tensors and train neural networks and GNNs.

The following resources might help you familiarize with PyTorch and PyTorch geometric.

* [PyTorch: Learn the Basics](https://pytorch.org/tutorials/beginner/basics/intro.html)
* [PyTorch geometric: Official tutorials](https://pytorch-geometric.readthedocs.io/en/latest/get_started/colabs.html#official-examples)


## Part 0: Explore the data [0 points]

This part contains no questions, but we will go together through the data to get a feeling of their content.
We work with the [GitHub dataset](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.GitHub.html), from the ["Multi-scale Attributed Node Embedding"](https://arxiv.org/abs/1909.13021) paper.
In this dataset, nodes represent developers on GitHub and edges are mutual follower relationships. It contains 37,300 nodes, 578,006 edges, 128 node features and 2 classes.

This data is readily available in PyTorch Geometric, let's start by installing it.

In [104]:
!pip install torch_geometric -q
!pip install pyg_lib -f https://data.pyg.org/whl/torch-2.2.0+cu121.html -q
!pip install torchmetrics -q

ERROR: Could not find a version that satisfies the requirement pyg_lib (from versions: none)
ERROR: No matching distribution found for pyg_lib


Then, we can import all relevant libraries. Some of them will be useful in later steps.

In [105]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import torch
import torch_geometric as pyg
from scipy import sparse
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from torch_geometric.datasets import GitHub
from torchmetrics import Metric
from torchmetrics.classification import Accuracy, BinaryF1Score, Precision, Recall

In [106]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Let's download the data.

In [107]:
dataset = GitHub(".")
data = dataset._data

Now, we shall study its content. Node attributes are accessible through the `x` attribute, which is a `torch.Tensor`.

In [108]:
print("Design matrix")
n_nodes, n_feats = data.x.shape
print(f"Num. nodes: {n_nodes}; num features: {n_feats}")

Design matrix


AttributeError: 'NoneType' object has no attribute 'shape'

We see that we have 37,700 nodes, each with 128 features. The features correspond to an embedding of location, starred repositories, employer and e-mail address of each user.

Each node comes with a 0/1 label, which indicates whether it corresponds to a web, or a machine learning developer.

In [ ]:
print("Target vector")
print("First five elements:", data.y[:5])
print("Number of samples:", data.y.shape)
print("Number of nodes in class 1:", data.y.sum().item())

We see that the task is quite imbalanced, as class one is underrepresented. To get more meaningful interpretations, we swap classes zero and one.

In [ ]:
data.y = 1 - data.y

The edges are contained in the `edge_index` attribute, which is again a tensor. Let's check its shape.

In [ ]:
print("Edge index shape:", data.edge_index.shape)

**0.1 [0 points]** Describe the content of the edge index matrix and how it relates to the adjacency matrix.

**Your answer here:**
...

Now, we will create two random binary masks on the nodes: one for training and one for testing. We would like to have 70% of the samples in the training split, so we will uniformly pick nodes with that probability.

We use a masking strategy instead of directly splitting the data because our interpretation of the task is that we have a social network in which the training label are accessible, while the test nodes, even though available, are unknown. This simplifies the sampling strategy, in particular for network methods, as we do not have to worry about loosing structure.

In [ ]:
rng = torch.Generator().manual_seed(452)
train_mask = torch.randn(n_nodes, generator=rng) < 0.8

n_nodes_tr = train_mask.sum().item()
print(f"Training set size: {n_nodes_tr} ({n_nodes_tr / n_nodes:.2%})")
print(f"Test set size: {n_nodes - n_nodes_tr} ({1 - n_nodes_tr / n_nodes:.2%})")
print(
    f"Ratio of class 1 in training: {torch.sum(train_mask * data.y).item() / n_nodes_tr:.2%}"
)

We saw that the graph has 37,700 nodes, which means that the dense adjacency matrix has 1,421,290,000 entries.
Supposing that binary variables are stored in a single bit, this would still require ~170 MB to store. With 8bit integers or floats it would occupy more than 10 GB, but with mainly zero values.

Since each node can fit in 16 bits, this representation can fit in 2.2 MB.

## Part 1: Deep learning on graph data [45 points]

This part presents a general workflow for deep learning and our recommended libraries: PyTorch and PyTorch Geometric.
We will start with classical ML baselines, to get some robust results to which we can compare. Then we will introduce Network features, to see whether they can help in our task. Finally, we start working with deep learning, and graph neural networks.

### Question 1.1: Our first baseline (4 points)

In this question we define a baseline model with a "classical" ml method, namely a random forest, to get an idea of what performances we can expect from the following models.
This model will only use node features, so it does not leverage at all the graph structure.

**1.1.1 [2pts]** Train a random forest classifier based on the node features. Make sure to use the provided `train_mask` for both the features and the target labels.

In [ ]:
# Your solution here ###########################################################

rf_classifier = ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.1.2 [1pts]** Predict the labels of the test nodes, then print the `classification_report`.

In [ ]:
# Your solution here ###########################################################


# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.1.3 [1pt]** Discuss which of the metrics is the most informative one for our setting.

**Your answer here:**
...

### Question 1.2: Graph baseline - Laplacian eigenmaps (5 points)

Now, let's implement a second benchmark, this time relying on structural properties. We would like to use eigenmaps of the Laplacian, but if you try to do it you would quickly run out of memory! (Go ahead and try if you will 😉)

The adjacency matrix is too big to use it in computations, but it would be mainly filled with zeros. We can optimize memory and running time by using a **sparse representation**.

**1.2.1 [1pts]** Compute the Laplacian matrix as a [sparse SciPy array][scipy_sparse]. Start by creating a sparse adjacency matrix form the `edge_index`, supposing that all edge weights are 1.

[scipy_sparse]: https://docs.scipy.org/doc/scipy/reference/sparse.html

In [ ]:
# Your solution here ###########################################################

adjacency = ...
laplacian = ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.2.2 [1pts]** Use SciPy sparse linear algebra capabilities to compute the first 5 nontrivial eigenvectors of the Laplacian.

*Note: This takes ~15 minutes, so you can change the condition to False after completing Question 1.2 to iterate more quickly over the following ones.

In [ ]:
# Your solution here ###########################################################

if True:  # Change to True to run cell

    eigvecs = ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

else:
    eigvecs = np.random.rand(n_nodes, 5)

**1.2.3 [1pts]** Train and test a new random forest classifier using the eigenvector representation as features.

In [ ]:
# Your solution here ###########################################################

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.2.4 [1pts]** Now, combine the two sets of features, i.e. the given programmers features and the Laplacian eigenmaps, into a single design matrix, then train and test another RF.

In [ ]:
# Your solution here ###########################################################

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.2.5 [1pts]** Comment on the results and describe which model you expect to perform the best on unseen dataset.



**Your answer here:**
...

### Question 1.3: Neural Network baseline - MLP (16 points)

In this question, we move from classical ML to deep learning and, again, we start from a simple model to get a viable benchmark.

**1.3.1 [2pts]** Create two `DataLoaders`, for the training and test data respectively, by using the `TensorDataset` class. Use the predefined batch size and shuffle training data.

References:
- [Datasets and DataLoaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)
- [PyTorch data utility](https://pytorch.org/docs/stable/data.html)

In [ ]:
batch_size = 128

# Your solution here ###########################################################

loader_train = ...

loader_test = ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.3.2 [2pts]** Define a [torch module][nn_module] for a two-layer perceptron with the ReLU activation function. The hidden dimension will be a parameter of the constructor function.
This neural network will take as input a design matrix and predict the "logits" of class 1.

[nn_module]: https://pytorch.org/docs/stable/generated/torch.nn.Module.html

In [ ]:
class MLP(nn.Module):
    # Your solution here #######################################################
    def __init__(self, in_features: int, hidden_features: int): ...

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.3.3 [1pts]** Define a function to perform one training step for a given NN, taking as argument a batch of data `x` with target `y`, an [optimizer](torch_optim), and a [loss function](torch_loss). The function should return the loss value, as a float.

In [ ]:
def train_nn_step(
    optimizer: optim.Optimizer,
    loss_fn: nn.Module,
    model: nn.Module,
    x: torch.Tensor,
    y: torch.Tensor,
) -> float:
    model.train()  # Used to ensure that relevant blocks are in training mode

    # Your solution here #######################################################
    ...
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.3.4 [2pts]** Write an evaluation function that takes as input a PyTorch Module, a data loader, and a [TorchMetrics function][torchmetrics] and returns the cumulative metric over all batches.

[TorchMetrics][torchmetrics] is a convenient package that implements metrics that work with PyTorch Tensors, and also with batched data.

[torchmetrics]: https://lightning.ai/docs/torchmetrics/stable/pages/quickstart.html

In [ ]:
def eval_nn(model: nn.Module, loader: DataLoader, metric_fn: Metric) -> float:
    model.eval()  # Used to ensure that relevant block are in evaluation model

    # Your solution here #######################################################
    ...
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.3.5 [2pts]** Create an instance of the previously defined MLP, a relevant [loss function][torch_loss] for our classification task, and an [optimizer](torch_optim).
When needed, as for the MLP hidden dimension and optimizer learning rate, select parameters that provide good results for the task. You might need some trial-and-error, so keep track of you results as you will be asked to comment on those hyperparameters.

Make sure to send everything to the correct device at initialization, as moving information from the CPU to the GPU is time-consuming.
To use GPUs, you might have to change runtime type.

[torch_loss]: https://pytorch.org/docs/stable/nn.html#loss-functions
[torch_optim]: https://pytorch.org/docs/stable/optim.html


In [ ]:
print(f"Using {device} device")

# Your solution here ###########################################################

mlp = ...

loss_fn = ...

optimizer = ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.3.6 [3pts]** Perform 10 epochs of training. During each epoch, you should perform training steps iterating over the whole dataset. Gather the losses of each batch, and plot the evolution of the training loss at the end.

In [ ]:
n_epochs = 15

# Your solution here ###########################################################

...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
plt.show()

**1.3.7 [1pts]** Evaluate the trained model on both the training and test data, using the most relevant metric from those already imported from TorchMetrics.

In [ ]:
# Your solution here ###########################################################

metric_fn = ...

metric_tr = ...
metric_te = ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

print(f"Training metric: {metric_tr:.3f}")
print(f"Test metric:     {metric_te:.3f}")

**1.3.8 [3pts]** Try different hyperparameters' combinations, in particular for the hidden dimension of the MLP and the learning rate of the optimizer. Then discuss the obtained results and the learning curves.

**Your answer here:**
...

### Question 1.4: Graph Neural Networks (20 points)

We will now shift from the standard deep learning paradigm to Graph Neural Networks, to leverage the additional structure of our data.

We already imported [PyTorch Geometric][torch_geometric] as `pyg`, so you can access its submodules as `pyg.nn`, `pyg.data` and so on.

[torch_geometric]: https://pytorch-geometric.readthedocs.io/en/latest/index.html

**1.4.1 [2pts]** Let's start by defining our first GNN. Again, it will be a subclass of the PyTorch `Module`, but this time it will take into account the `edge_index` in its `forward method`. Use two [GCN layers][gcn] to go from input features, here called *channels*, to a hidden dimension defined in the constructor, then to logit readout. Use ReLU activations.

This GNN will map node vectors to node logits, so we can directly read out node probabilities from

[gcn]: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GCNConv.html

In [ ]:
class GCN(nn.Module):
    # Your solution here #######################################################
    def __init__(self, in_channels: int, hidden_channels: int): ...

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.4.2 [3pts]** Perform `n_epochs` of training of a GCN model with 64 hidden channels, using full training data as a batch. Make sure to only use training data in the loss computation by using the `train_mask`. Track the loss value at each step and plot it. Finally, evaluate the model on train and test, using the `metric_fn` from before.

In [ ]:
# Your solution here ###########################################################

gcn = ...

metric_tr, metric_te = ..., ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
plt.show()

print(f"Training metric: {metric_tr:.3f}")
print(f"Test metric:     {metric_te:.3f}")

**1.4.3 [2pts]** Hopefully, we got already some good results, but we would like to test whether stochastic optimization might be better. Batching graph data requires a particular approach, since on top of the design matrix with node features we have to account for edge information. In our setting, we have a single graph with many nodes, and a node level task. A batching strategy consists in sampling nodes with their neighbors, then working with this smaller graphs in a batched way.

Define one [NeighborLoader][neighborloader] for the training data, which will gather neighbors for as many *iterations* as layers in your GCN.

References:
- [Mini batches](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html#mini-batches)

[neighborloader]: https://pytorch-geometric.readthedocs.io/en/latest/modules/loader.html#torch_geometric.loader.NeighborLoader

In [ ]:
batch_size = 1024

# Your solution here ###########################################################

loader_graph_train = ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.4.4 [1pt]** Use the previously defined `train_nn_step` to train a newly initialized GCN with the new loader. Again plot the loss evolution and evaluate the trained model on train and test data.

In [ ]:
# Your solution here ###########################################################


# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
plt.show()

**1.4.5 [1pts]** Predict the label probabilities of each node and evaluate.

In [ ]:
# Your solution here ###########################################################

metric_tr, metric_te = ..., ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

print(f"Training metric: {metric_tr:.3f}")
print(f"Test metric:     {metric_te:.3f}")

**1.4.6 [2pts]** Define a new GNN architecture using [graph attention layers][gat].

[gat]: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GATv2Conv.html#torch_geometric.nn.conv.GATv2Conv

In [ ]:
class GAT(nn.Module):
    # Your solution here #######################################################
    def __init__(self, in_channels: int, hidden_channels: int): ...

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.4.7 [3pts]** Train and evaluate the GAT model with both methods.

In [ ]:
print("FULL TRAINING")

# Your solution here ###########################################################

metric_tr, metric_te = ..., ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
plt.title("Full training")
plt.show()

print(f"Training metric: {metric_tr:.3f}")
print(f"Test metric:     {metric_te:.3f}")

print("BATCH TRAINING")

# Your solution here ###########################################################

metric_tr, metric_te = ..., ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

plt.title("Full training")
plt.show()

print(f"Training metric: {metric_tr:.3f}")
print(f"Test metric:     {metric_te:.3f}")

**1.4.8 [6pts]** Compare the results of these two architectures, with multiple hyperparameters, and the previous results. Discuss the eventual differences in performance highlighting the properties that you believe influence most the outcome.

**Your answer here:**
...

## Part 2: Learning graphs [20 points]

Graph attention layers are quite interesting, since they use the local and the incoming information of a node to give different weights to each neighbor. This is conceptually similar to learning a new graph on top of the existing one.

In this part, we design a block that, from node embeddings, will produce a new graph. The idea is similar to the one in the paper "Discrete Graph Structure Learning for Forecasting Multiple Time Series", which is illustrated in the following Figure.

References:
- C. Shang, J. Chen, and J. Bi, “Discrete Graph Structure Learning for Forecasting Multiple Time Series,” presented at the International Conference on Learning Representations, Feb. 2022. Accessed: Aug. 15, 2022. Available: https://openreview.net/forum?id=WEHSlH5mOk




![Graph Learning Module](graph_learning_module.png)

As we can see from the schema, we have three main components:
1. **Feature extractor**: mapping each node to a new, synthesized representation;
2. **Link predictor**: for each pair of node representations, predict the probability that an edge links them. We gather probabilities in a *structure matrix* $\theta$;
3. **Sampling**: Sample one, or multiple, discrete graphs from the structure matrix.

In the following questions, we will break down these components.

### Question 2.1: Sampling (4 points)

Sampling is the most intriguing part of our module, as it maps, randomly, continuous probabilities to discrete edges.
Ideally, we would like to sample each edge with a probability $\theta$, following a Bernoulli distribution, but this would be hard to backpropagate through.

What we do instead is known as the **Gumbel Trick**.
We sample edges using a [Gumbel][gumbel] reparameterization, which allows differentiating for $\theta$ through it. With $g_{ij}^1, g_{ij}^2 \sim \operatorname{Gumbel}(0,1)$ for all $i,j$, and $s$ a temperature parameter,
$$
A_{ij} = \operatorname{sigmoid}\left(
  \frac{
    \log\left( \frac{\theta_{ij}}{1 - \theta_{ij}} \right)
    + g_{ij}^1 - g_{ij}^2
  }{s}
\right)
.
$$
By letting the temperature go to zero, we can get closer and closer to a Bernoulli distribution.

[gumbel]: https://en.wikipedia.org/wiki/Gumbel_distribution


**2.1.1 [2pts]** Define a function to sample a matrix of Gumbel variables of given shape, knowing that, for $p$ sampled uniformly in (0,1), then $Q(p) \sim \operatorname{Gumbel}(\mu,\beta)$
$$
  Q(p)=\mu-\beta \ln (-\ln (p))
  .
$$

In [33]:
import torch

def sample_gumbel(shape, mu=0, beta=1):
    # Your solution here #######################################################
    
    # Populate matrix of dimensions 'shape' by sampling uniformly between (0, 1)
    uniform_samples = torch.rand(shape)

    # Apply the Gumbel inverse CDF transformation using our uniformly sampled matrix
    gumbel_samples = mu - beta * torch.log(-torch.log(uniform_samples))

    return gumbel_samples

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

print("Testing sample_gumbel")
print(sample_gumbel((2, 3), 0, 1))

Testing sample_gumbel
tensor([[2.3862, 1.4038, 0.5967],
        [0.4790, 0.6078, 0.1385]])


**2.1.2 [2pts]** Note that $log(\frac{\theta}{1 - \theta})$ is the sigmoid function, so we can work with unnormalized edge logits instead of probabilities. Define a function to sample an adjacency matrix $A$ from edge logits using the Gumbel Trick.

In [34]:
def sample_gumbel_trick(logits, temperature, mu=0, beta=1):
    # Your solution here #######################################################
    
    # Populate matrices of dimensions logits.shape by sampling from gumbel distribution
    g1 = sample_gumbel(logits.shape, mu, beta)
    g2 = sample_gumbel(logits.shape, mu, beta)

    # Apply Gumbel trick to sample adjacency matrix that emulates Bernoulli draw (to varying degrees based on temperature)
    return torch.sigmoid((logits + g1 - g2) / temperature)
    
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


print("Testing sample_gumbel_trick")
print(sample_gumbel_trick(torch.tensor([1000, 0, 0, -10]), temperature=10))
print(sample_gumbel_trick(torch.tensor([1000, 0, 0, -10]), temperature=1))
print(sample_gumbel_trick(torch.tensor([1000, 0, 0, -10]), temperature=1e-2))

Testing sample_gumbel_trick
tensor([1.0000, 0.4816, 0.5830, 0.2792])
tensor([1.0000e+00, 5.1260e-02, 2.6817e-01, 6.9106e-06])
tensor([1., 0., 1., 0.])


### Question 2.2: Link predictor (11 points)

GNNs are all about node embeddings, which by now we should know how to deal with. The missing component is therefore the **link predictor**.

Naively, we could iterate through all pairs of nodes and apply a predictor layer, but it would be highly inefficient.
To leverage tensor manipulation, let's start by gathering paired node representations in a matrix, so that we can predict probabilities in parallel.

**2.2.1 [3pts]**  Define a function that takes as input a tensor of node embeddings, and returns a tensor that concatenate embeddings pairwise. Use [triu_indices][triu_indices] to have pairs appearing only once and avoid self loops, and return the indices along with the embeddings.

[triu_indices]: https://pytorch.org/docs/stable/generated/torch.triu_indices.html

In [35]:
def pair_embeddings(x) -> (torch.Tensor, torch.Tensor):
    # Your solution here ###########################################################
    
    # confim input tensors are 2D (non-batch) or 3D (batch)
    assert x.dim() in [2,3]
    
    # non-batched case
    if x.dim() == 2:
        # num nodes in non-batch input
        num_nodes = x.shape[0]
        # non-batch input doesn't get batch index
        batch_indices = None
        # index combinations, third argument ensures we are above diagonal (so we don't pair a node with itself)
        row_indices, col_indices = torch.triu_indices(num_nodes, num_nodes, 1)
        # concat embeddings
        paired_embeddings = torch.cat((x[row_indices], x[col_indices]), dim=1)
    # batched case
    else:
        # Now we have batches - first dimension is batch index 
        batch_size, num_nodes, _ = x.shape
        # create an index to track the batch number of the output pairs
        batch_indices = torch.arange(batch_size).repeat_interleave(num_nodes)
        # index combinations, third argument ensures we are above diagonal (so we don't pair a node with itself)
        # these will be used for each batches
        sing_batch_row_indices, sing_batch_col_indices = torch.triu_indices(num_nodes, num_nodes, 1)
        # broadcast single-batch indices
        # (row/col)_indices goes from length num_nodes to length num_nodes * batch_size
        row_indices, col_indices = sing_batch_row_indices.repeat(batch_size), sing_batch_col_indices.repeat(batch_size) 
        # concat embeddings across all batches at once
        paired_embeddings = torch.cat((x[batch_indices, row_indices], x[batch_indices, col_indices]), dim=1)

    return paired_embeddings, (row_indices, col_indices, batch_indices)

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

print("Testing pair_embeddings")
print(pair_embeddings(torch.tensor([[1.0], [2.0], [3.0]]))[0])
print(pair_embeddings(torch.tensor([[[1.0], [2.0], [3.0]], [[4.0], [5.0], [6.0]]]))[0])

Testing pair_embeddings
tensor([[1., 2.],
        [1., 3.],
        [2., 3.]])
tensor([[1., 2.],
        [1., 3.],
        [2., 3.],
        [4., 5.],
        [4., 6.],
        [5., 6.]])


**2.2.2 [8pts]** Define a PyTorch module that takes as input node embeddings, compute link probabilities with a two-layer perceptron on paired embeddings, then samples edges with the Gumbel trick. The output of the forward method will be a PyTorch Geometric [EdgeIndex][edge_index] of tensors representing indices and weights corresponding to positively sampled edges. You might need a `eps` threshold to avoid numerical errors.

[edge_index]: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.edge_index.EdgeIndex

In [103]:
import torch
import torch.nn as nn
import torch_geometric as pyg

class MLPGraphLearn(nn.Module):
    # Your solution here ###########################################################
    def __init__(
        self, in_features: int, hidden_features: int, temperature: float, eps=1e-10
    ):
        super().__init__()

        # layers for two-layer perceptron
        # fc1: in_features * 2 because inputs into fc1 are concetenated embeddings for pairs
        # fc2: output is size 1 because we are seeking probability of pair connection
        self.fc1 = nn.Linear(in_features*2, hidden_features)
        self.fc2 = nn.Linear(hidden_features, 1) 

        # init other parameters (temperature for gumbel trick, eps for numerical errors)
        self.temperature = temperature
        self.eps = eps

    
    def forward(self, x):

        # compute pairwise embeddings (function from 2.2.1)
        paired_embeddings, (row_indices, col_indices, batch_indices) = pair_embeddings(x)

        # feed embeddings through two-layer perceptron
        # relu activation between fc1 and fc2
        fc1_out = torch.relu(self.fc1(paired_embeddings))
        logits = self.fc2(fc1_out).squeeze(-1)

        # Gumbel trick to sample edges (function from 2.1.2)
        gumbel_adj_matrix = sample_gumbel_trick(logits, temperature=self.temperature)

        # Retain edges with >0.5 probability in gumbel trick adjacency matrix
        edge_mask = gumbel_adj_matrix > 0.5
        
        # pull predicted edges with edge mask, create EdgeIndex object
        # graph is assumed to be undirected because 2.2.1 asks us to include each pairs just once
        predicted_edges = torch.stack([row_indices[edge_mask], col_indices[edge_mask]], dim=0)
        edge_indices = pyg.EdgeIndex(predicted_edges, is_undirected=True)

        # pull predicted edge weights with edge mask
        weights = gumbel_adj_matrix[edge_mask]

        # uncomment print statement to ensure proper edges are retained
        # element-wise pairs from row_indices, col_indices should appear in function output if > 0.5 in gumbel_adj_matrix
        # also, only positively sampled edge weights should appear
        # print(row_indices, col_indices, gumbel_adj_matrix, weights)

        # based on description, I believe we return two objects: EdgeIndex and weight tensor. 
        # I previously interpreted as 1 object, but you can't add weights to EdgeIndex
        # Also, 'weights' is unclear - current guess is gumbel-sampled adjacency matrix values
        return edge_indices, weights

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

print("Testing MLPGraphLearn")
mlp_gl = MLPGraphLearn(2, 5, 0.01)
mlp_gl(torch.tensor([[[1.0, 2.0], [0.5, 7.1], [-0.1, 0.3]]]))

Testing MLPGraphLearn


(EdgeIndex([[0, 1],
            [2, 2]], nnz=2, is_undirected=True),
 tensor([1., 1.], grad_fn=<IndexBackward0>))

### Question 2.3: Classifiers with Graph Learning Module (5 points)

Let's introduce our graph learning block into some classifiers.


**2.3.1 [4pts]** Define a classifier that first produces node embeddings with a Linear layer with ReLU activation, which it feeds to the previously defined GL module; then it performs two graph convolutions on the original node features using the learned graph.

In [ ]:
from torch_geometric.data import Data
import torch_geometric as pyg

class MLPGLClassifier(nn.Module):
    # Your solution here #######################################################
    def __init__(
        self,
        in_features: int,
        gl_node_features_in: int = 64,
        gl_node_features_hidden: int = 32,
        gcn_hidden: int = 64,
    ):
        super().__init__()

        # linear layer for embeddings
        self.fc = nn.Linear(in_features, gl_node_features_in)

        # GL module for learned graph
        self.gl = MLPGraphLearn(gl_node_features_in*2, gl_node_features_hidden, temperature=0.1)

        # convolutional layers for convolution 'on the original node features using the learned graph'
        self.conv1 = pyg.nn.GraphConv(in_features, gcn_hidden)
        self.conv2 = pyg.nn.GraphConv(gcn_hidden, 64)

    def forward(self, x):

        # create embeddings
        fc_out = torch.relu(self.fc(x))

        # GL module
        predicted_EI, weights = self.gl(fc_out)

        # instantiate learned graph
        learned_graph = Data(x=weights, edge_index=predicted_EI)
        
        # perform convolutions
        conv1_out = self.conv1(x, predicted_EI).relu()
        conv2_out = self.conv2(conv1_out, predicted_EI).relu()

        # we have no info on the classification being made???


    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**2.3.2 [1pt]** Unfortunately, training and evaluating the `MLPGLClassifier` might take too long. Let's just test whether it works: instantiate the classifier and compute the graph embedding.

In [ ]:
# Your solution here ###########################################################

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

## Part 3 (Theory): Forcing causal structure in learned graphs [35 points]

Understanding and mapping the causal relationships among data variables, represented by directed acyclic graphs (DAGs), presents a significant challenge. The search space for these DAGs is combinatorial, and it scales super exponentially with the number of nodes, further complicating the task.

Assuming causal relationships between data variables, the basic DAG learning problem is formulated as follows: Let $\mathbf{X} \in \mathbb{R}^{n \times d}$ be a data matrix consisting of $n$ i.i.d. observations of the random vector $x=\left(x_1, \ldots, x_d\right)$ and let $\mathbb{D}$ denote the (discrete) space of DAGs $\mathrm{G}=(\mathrm{V}, \mathrm{E})$ on $d$ nodes. Given $\mathbf{X}$, we seek to learn a DAG $\mathrm{G}$ represented by its adjacency matrix $A$ such that:

$$
\begin{array}{rl}
\min _{A \in \mathbb{R}^{d \times d}} &  F(A):= \frac{1}{2 n}\|\mathbf{X}-GNN(\mathbf{X}, A)\|_F^2+\lambda\|A\|_1 \\
\text { subject to } &  A \in \mathbb{D} ,
\end{array}
$$
where $\mathbb{D}$ represents a DAG space and GNN is a graph neural network that simultaneously learns a DAG $A$ and accurately predicts an estimation of the matrix $\mathbf{X}$. The minimization of the aforementioned problem guides us towards finding the causal graph that generates the data $\mathbf{X}$. Although $F(A)$ is continuous, the DAG constraint $A \in \mathbb{D}$ remains a challenge to enforce.

The objective is to make the aforementioned problem amenable to black-box optimization (in order to use SGD, ADAM...). We aim to replace the combinatorial acyclicity constraint $A \in \mathbb{D}$ with a single smooth equality constraint $h(A) = 0$. Thus, the objective of this section is to find a smooth function $h: \mathbb{R}^{d \times d} \rightarrow \mathbb{R}$ that will satisfy the following: $h(A)=0$ if and only if $A$ is acyclic (i.e. $A \in \mathbb{D}$). Furthermore, we want to ensure that $h$ and its derivatives are easy to compute.

### Question 3.1: DAGness property for binary matrix with spectral radius condition (8 points)

In this question we want to find out when a matrix $A \in\{0,1\}^{d \times d}$ corresponds to an acyclic graph.

Suppose $A \in\{0,1\}^{d \times d}$ and $r(A)<1$, $r(A)$ is the spectral radius of the matrix $A$, and it corresponds to the largest absolute eigenvalue of $A$.



**3.1.1 [2pts]** Justify the convergence of $\sum_{k=0}^{\infty} A^k$ and show that:
$$\left(I_n-A\right)^{-1}=\sum_{k=0}^{\infty} A^k$$

**Your answer here:**

We have that the largest absolute eigenvalue of A is less than 1. This implies all eigenvalues of A have magnitude less than 1. Letting $\lambda _{1}, \lambda _{2}, ..., \lambda _{n}$ be the eigenvalue set $E$ of $A$, the eigenvalues of $A^k$ must equal $\lambda _{1}^k, \lambda _{2}^k, ..., \lambda _{n}^k.$ But $\lim_{{k \to \infty}}\lambda_i^k = 0 \ \forall \ \lambda_i \in E$, meaning all eigenvalues for square matrix $A^k$ converge to 0 as ${k \to \infty}$. $A^k$ therefore converges to a nilpotent matrix as ${k \to \infty}$, which in turn implies the terms of $\sum_{k=0}^{\infty}A^k$ must converge to the 0 matrix. Because all constituent entries in the matrix power series are linear combinations of 0s and 1s, and because $\lim_{{k \to \infty}}A^k$ converges to the 0 matrix, $\sum_{k=0}^{\infty} A^k$ converges.

We now show $\left(I_n-A\right)^{-1}=\sum_{k=0}^{\infty} A^k$ (proof based on the Neumann series [wiki](https://en.wikipedia.org/wiki/Neumann_series)):

Let $P_i = \sum_{k=0}^{i}A^k$. Then:

$(I_n - A)\lim_{{i \to \infty}}P_i = \lim_{{i \to \infty}}(I_n-A)P_i = \lim_{{i \to \infty}}(P_i - AP_i) = $

$\lim_{{i \to \infty}}(\sum_{k=0}^{i}A^k - \sum_{k=0}^{i}A^{k+1}) = \lim_{{i \to \infty}}(I_n + \sum_{k=1}^{i}A^k - \sum_{k=1}^{i}A^{k} - A^{k+1}) = \lim_{{i \to \infty}}(I_n - A^{k+1}).$

We established above that $\sum_{k=0}^{\infty} A^k$ converges, which means as $k$ approaches infinity, the entries of $A^{k+1}$ must converge to 0. Thus:

$\lim_{{i \to \infty}}(I_n - A^{k+1}) = (I_n - \bf{0}) = I_n.$

But notice $\lim_{{i \to \infty}}P_i = \sum_{k=0}^{\infty} A^k$, meaning we have shown $(I_n - A)\sum_{k=0}^{\infty} A^k = I_n$. This in turn implies $\sum_{k=0}^{\infty} A^k = (I_n - A)^{-1}.$


**3.1.2 [6pts]** Show that $A$ is a $D A G$ if and only if $\operatorname{tr}(I-A)^{-1}=d$

**Your answer here:**

We have from 3.1.1 that $\left(I_n-A\right)^{-1}=\sum_{k=0}^{\infty} A^k$.

Thus, $\operatorname{tr}((I-A)^{-1})=\operatorname{tr}(\sum_{k=0}^{\infty} A^k)$ -> linearity of summation -> $\sum_{k=0}^{\infty}\operatorname{tr}(A^k) = \operatorname{tr}(A^0) + \sum_{k=1}^{\infty}\operatorname{tr}(A^k)$

Since $A \in\{0,1\}^{d \times d}$, $\operatorname{tr}(A^0) = \operatorname{tr}(I \in \{0,1\}^{d \times d}) = d$.

So $\operatorname{tr}(I-A)^{-1} = d + \sum_{k=1}^{\infty}\operatorname{tr}(A^k) = d + \sum_{k=1}^{\infty}\sum_{i=1}^{d}{A^k}_{ii}$.

But an adjacency matrix describes a DAG iff there are 0 cycles - that is, iff there are 0 directed paths of any length between a node and itself. This is equivalent to saying ${A^k}_{ii} = 0$ $\forall k > 0$ (see Lecture 1 - Graph Theory basics, slide 42). 

Hence, an adjacency matrix A describes a DAG iff $\sum_{k=1}^{\infty}\sum_{i=1}^{d}{A^k}_{ii} = 0.$

If follows that an adjacency matrix A is a DAG iff $\operatorname{tr}(I-A)^{-1} = d + \sum_{k=1}^{\infty}\sum_{i=1}^{d}{A^k}_{ii} = d + 0 = d$.

--

source: https://arxiv.org/pdf/1803.01422

Having an assumption on $r(A) < 1$ limits the application of our results. For this reason, our objective is to generalize the previous result to binary matrices.


### Question 3.2: DAGness property for binary matrix (8 points)

Suppose $A \in\{0,1\}^{d \times d}$.

**3.2.1 [4pts]**  Prove the existence of the exponential matrix, we recall that $e^{A} = \sum_{k=0}^{\infty} \frac{A^k}{k !}$


**Your answer here:**

Suppose to the contrary that for some adjacency matrix $A \ \nexists \ e^{A} = \sum_{k=0}^{\infty} \frac{A^k}{k !} $. 

Then for some $k$, either $k ! = 0$, $k !$ is undefined, or $A^k$ is undefined.

But since $k \geq 0$ and $k \in \mathbb{N}$, $k! \geq 1$ and $k! \in \mathbb{Z} \ \forall \ k$. Hence $k !$ is always defined and can never equal 0.

Furthermore, since $A \in\{0,1\}^{d \times d}$, we know that regardless of $k$ every product $A^k$ will multiply successive matrices of dimensions ${d \times d}$ - meaning matrix multiplication can always occur - and all values in $A^k$ will be in $\mathbb{N}$ (as they will be linear combinations of 0s and 1s). Hence, $A^k$ is always defined.

Since neither $k !$ nor $A^k$ can be undefined and $k !$ cannot equal 0, there cannot exist a $k$ for which $\frac{A^k}{k !}$ is undefined, and we have reached a contradiction. This means $e^{A} = \sum_{k=0}^{\infty} \frac{A^k}{k !}$ exists for all A.

**3.2.2 [4pts]** Show that $A$ is a $D A G$ if and only if $\operatorname{tr} e^A=d$, where $e^A$ is the exponential matrix.

**Your answer here:**

We have from 3.2.1 that $e^{A} = \sum_{k=0}^{\infty} \frac{A^k}{k !}$.

Thus, $\operatorname{tr}(e^{A})=\operatorname{tr}(\sum_{k=0}^{\infty} \frac{A^k}{k !})$ -> linearity of summation -> $\sum_{k=0}^{\infty}\operatorname{tr}(\frac{A^k}{k !}) = \operatorname{tr}(A^0) + \sum_{k=1}^{\infty}\operatorname{tr}(\frac{A^k}{k !})$

Since $A \in\{0,1\}^{d \times d}$, $\operatorname{tr}(A^0) = \operatorname{tr}(I \in \{0,1\}^{d \times d}) = d$.

So $\operatorname{tr}(e^{A}) = d + \sum_{k=1}^{\infty}\operatorname{tr}(\frac{A^k}{k !}) = d + \sum_{k=1}^{\infty}\frac{1}{k !}\sum_{i=1}^{d}(A^k)_{ii}$.

As in 3.1.2, an adjacency matrix A describes a DAG iff ${A^k}_{ii} = 0$ $\forall k > 0$ (see Lecture 1 - Graph Theory basics, slide 42). 

Hence, an adjacency matrix A describes a DAG iff $\sum_{k=1}^{\infty}\frac{1}{k !}\sum_{i=1}^{d}{A^k}_{ii} = \sum_{k=1}^{\infty}\frac{1}{k !}(0) = 0$.

If follows that an adjacency matrix A is a DAG iff $\operatorname{tr}(e^{A}) = d + \sum_{k=1}^{\infty}\frac{1}{k !}\sum_{i=1}^{d}{A^k}_{ii} = d + 0 = d$.

--

source: https://arxiv.org/pdf/1803.01422

**3.3 [5pts]** DAGness property for weighted adjacency matrices

Suppose $A \in \mathbb{R}^{d \times d}$. Knowing that the equivalence of Question 3.2 is correct for any nonnegative weighted matrix $A$, propose a function $f$ such that $A$ is a $D A G$ if and only if $\operatorname{tr} e^{f(A)}=d$.


**Your answer here:**

Let $f(A) = A \odot A$. In other words, let $f(A)$ be the matrix that results from squaring each element of A; $(A \odot A)_{ij} = A_{ij}A_{ij}$.

We have from 3.2.2 and the 3.3 prompt that $\operatorname{tr} e^A=d$ provided A is nonnegative (ie each of its elements is nonnegative) iff A is a DAG. Meanwhile, $f(A)$ is nonnegative, as any nonpositive element of A becomes nonnegative when squared with itself. Furthermore, $A \in \mathbb{R}^{d \times d} \implies A \odot A \in \mathbb{R}^{d \times d}$ because the square of a real number is also a real number. Hence $ \exists \ f(A) \ \forall \ A $.

Replacing $A$ with $f(A)$ in the equality from 3.2.2:

$\operatorname{tr}(e^{f(A)})=\operatorname{tr}(\sum_{k=0}^{\infty} \frac{(A \odot A)^k}{k !})$ -> linearity of summation -> $\sum_{k=0}^{\infty}\operatorname{tr}(\frac{(A \odot A)^k}{k !}) = \operatorname{tr}((A \odot A)^0) + \sum_{k=1}^{\infty}\operatorname{tr}(\frac{(A \odot A)^k}{k !}) = d + \sum_{k=1}^{\infty}\operatorname{tr}(\frac{(A \odot A)^k}{k !}) = d + \sum_{k=1}^{\infty}\frac{1}{k !}\sum_{i=1}^{d}((A \odot A)^k)_{ii}$

It remains to prove $\sum_{k=1}^{\infty}\frac{1}{k !}\sum_{i=1}^{d}((A \odot A)^k)_{ii} = 0$.

Now, since $a^2_{ij} = 0$ iff $a_{ij} = 0$ and $a^2_{ij} \neq 0$ iff $a_{ij} \neq 0 \ \forall \ a_{ij} \in A$, $A$ and $A \odot A$ share the same edge sets (if not the same edge weights). $A = DAG \iff A \odot A = DAG$.

As referenced in 3.1.2 and 3.2.2, an adjacency matrix B describes a DAG iff ${B^k}_{ii} = 0$ $\forall k > 0$ (see Lecture 1 - Graph Theory basics, slide 42). 

Thus, since $A \odot A$ is a DAG, $\sum_{k=1}^{\infty}\frac{1}{k !}\sum_{i=1}^{d}((A \odot A)^k)_{ii} = \sum_{k=1}^{\infty}\frac{1}{k !}(0) = 0$, meaning $\sum_{k=1}^{\infty}\operatorname{tr}(\frac{(A \odot A)^k}{k !}) = 0$.

It follows that $f(A)$ is specified such that $A$ is a $D A G$ if and only if $\operatorname{tr}(e^{f(A)=A \odot A})=d$.

--

source: https://arxiv.org/pdf/1803.01422

**3.4 [7pts]** Compute the gradient of $h(A) = \operatorname{tr} e^{f(A)}-d$.

**Your answer here:**

If $h(A) = \operatorname{tr}(e^{A \odot A})-d$, then

$\nabla h(A) = \frac{\partial}{\partial A}\operatorname{tr}(e^{A \odot A}) - \frac{\partial}{\partial A}d = \frac{\partial}{\partial A}\operatorname{tr}(e^{A \odot A}) - 0 = \frac{\partial}{\partial A}\operatorname{tr}(e^{A \odot A}) = \frac{\partial}{\partial A}\sum_{i=1}^{d}{e^{A \odot A}}_{ii} = \sum_{i=1}^{d}\frac{\partial}{\partial A}{e^{A \odot A}}_{ii} = \langle I, \frac{\partial}{\partial A}{e^{A \odot A}} \rangle_F$.

Let $S = A \odot A$. Then:

$\langle I, \frac{\partial}{\partial A}{e^{A \odot A}} \rangle_F = \langle I, \frac{\partial}{\partial A}{e^{S}} \rangle_F = \langle I, \frac{\partial}{\partial S}{e^{S}}\frac{\partial}{\partial A}{S} \rangle_F = \operatorname{tr}(I^T\frac{\partial}{\partial S}{e^{S}}\frac{\partial}{\partial A}{S}) = \langle I(\frac{\partial}{\partial S}{e^{S}})^T,\frac{\partial}{\partial A}{S} \rangle_F = $<span style="color:red;">is this allowed?</span>$ = \langle I(e^{S})^T,\frac{\partial}{\partial A}{S} \rangle_F = \langle (e^{S})^T,\frac{\partial}{\partial A}{S} \rangle_F$.

Now, $\frac{\partial}{\partial A}{S} = \frac{\partial}{\partial A}{A \odot A} = A \odot \frac{\partial}{\partial A}A + \frac{\partial}{\partial A}A \odot A $ -> commutative, distributive -> $2A \odot \frac{\partial}{\partial A}A.$

Then $\nabla h(A) = \langle (e^{S})^T,2A \odot \frac{\partial}{\partial A}A \rangle_F$ -> mutual commutativity -> $\langle (e^{S})^T \odot 2A, \frac{\partial}{\partial A}A \rangle_F$.

But $\frac{\partial}{\partial A}A$ is a matrix with 1 for each element. So $\nabla h(A) = (e^{S})^T \odot 2A = (e^{A \odot A})^T \odot 2A$.

--

sources: 

general structure:
https://math.stackexchange.com/questions/4727042/on-the-weighted-adjacency-matrix-of-a-directed-acyclic-graph-dag

for tr(AB) = <A^T, B>:
https://math.stackexchange.com/questions/1898839/relation-between-frobenius-norm-and-trace

for commutative and distributive of hadamard: https://en.wikipedia.org/wiki/Hadamard_product_(matrices)

for gradient of hadamard product and mutual commutativity: https://math.stackexchange.com/questions/2307026/gradient-hadamard-product


**3.5 [7pts]** Other alternatives

Suppose $A \in \mathbb{R}^{d \times d}$. For the previously proposed function $f$ show that for any $\alpha > 0 $, $A$ is a $D A G$ if and only if $\operatorname{tr}\left[(I+\alpha f(A))^d\right]-d=0$.

**Your answer here:**

We must show A is a $DAG$ iff $\operatorname{tr}\left[(I+\alpha (A \odot A))^d\right]-d=0$.

Observe $\operatorname{tr}\left[(I+\alpha (A \odot A))^d\right]-d=0 \implies \operatorname{tr}\left[(I+\alpha (A \odot A))^d\right]=d.$

Because $A$ is an adjacency matrix and $f(A)$ does not add or remove edges, $(A \odot A)$ is an adjacency matrix that has the same edge set (but not edge weights) as $A$. Similarly, because $alpha > 0$, $\alpha(A \odot A)$ is an adjacency matrix that has the same edge set (but not edge weights) as $A$. Finally, in terms of the original graph, by adding $I$ to $\alpha(A \odot A)$, we are adding exactly $d$ edges to the original edge set: one self-loop at each node. Thus, the edge set for adjacency matrix $W = I+\alpha (A \odot A)$ contains all the same edges as A along with $d$ new self-loops.

We must now prove that A is a $DAG$ iff $\operatorname{tr}(W^d) = d$.

From 3.3 we have that $A \odot A$ is a $DAG$ iff $A$ is a $DAG$. By logic similar to that mentioned in 3.3, $\alpha (A \odot A)$ is a $DAG$ iff $A$ is a $DAG$ (because the scaling factor $\alpha > 0$ does not change whether an entry is zero or nonzero and therefore does not add or remove edges in the adjacency matrix). Therefore, $W = I + D_{DAG}$ iff $A$ is a $DAG$, where the $DAG$ $D_{DAG} = \alpha(A \odot A)$. $W$ itself is clearly not a $DAG$; that each node has a self-loop precludes this. 

We now claim that any cycle of arbitrary length $l$ starting and ending at node $n$ in $W$ must be composed of $l$ traversals of the single self-loop attached to $n$. To see why this is the case, suppose to the contrary that there is a cycle starting and ending at $n$ that traverses at least one non-self-loop edge. Then, because $I$ does not add any edges *between* nodes to $D_DAG$, the cycle must be relying on one or more edges from $D_DAG$ to eventually return to $n$. Furthermore, in this scenario, the path would still constitute a cycle from $n$ to $n$ if any self-loop traversals were removed since self-loop traversals do not change the active node on a path. This would mean the supposed path is (or has a subset that contains) a cycle that depends exclusively on edges from $D_DAG$. But we know $D_DAG$ is a $DAG$, and that such a path cannot exist. We have reached a contradiction. As a result, there can be no cycle of length $l$ that is not composed of $l$ traversals of a node's self-loop. Because each node in $W$ has just one self-loop edge due to $I$, this also means there are $d$ total cycles of length $l$ in W - one for each node.

Per slide 42 of Lecture 1 - Graph Theory basics, ${W^d}_{ij}$ is equal to the number of paths of length $d$ between nodes $i$ and $j$. Hence, according to the paragraph above, if $A$ is a $DAG$ then $\operatorname{tr}(W^d) = \sum_{k=1}^{d}{W^d}_{kk} = 1 + ... + 1 = d$. 

To see why the latter implies the former, suppose to the contrary that $\operatorname{tr}(W^d) = d$ but $W = I + E_{notDAG}$, where $E_{notDAG} = \alpha(A \odot A)$ is not a $DAG$ (meaning $A$ is necessarily not a $DAG$). Then there would exist a cycle $c$ of length $l \leq d$ in $E_{notDAG}$ for some node $n$. If $l = d$, it is obvious that both $c$ and the full self-loop cycle for $n$ induced by $I$ would be included in ${W^d}_{kk}$, meaning ${W^d}_{kk} > 1$ and $\operatorname{tr}(W^d) > d$. If $l < d$, one could add $d-l$ self-loop traversals to the cycle $c$ to increase its length, increment ${W^d}_{kk}$, and cause $\operatorname{tr}(W^d) > d$. By contradiction in both cases, we have that $\operatorname{tr}(W^d) = d$ iff $A$ is a $DAG$ (where $W = I+\alpha (A \odot A))$. 